In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

np.random.seed(7)

In [ ]:
dest = '/content/drive/MyDrive/HSE Final Project/to_train'
name = 'submit_9_abandoned.csv'

df = pd.read_csv(dest + '/' + name)
df.shape

(10913804, 43)

In [ ]:
df['shop_id'] = df['shop_id'].astype(np.int8)
df['item_id'] = df['item_id'].astype(np.int16)
df['month'] = df['month'].astype(np.int8)
df['year'] = df['year'].astype(np.int16)
df['date_block_num'] = df['date_block_num'].astype(np.int8)
df['item_cnt'] = df['item_cnt'].astype(np.int32)
df['prev_value'] = df['prev_value'].astype(np.int32)
df['prev_value_year'] = df['prev_value_year'].astype(np.int32)
df['shop_city'] = df['shop_city'].astype(np.int8)
df['shop_category'] = df['shop_category'].astype(np.int8)
df['item_category_id'] = df['item_category_id'].astype(np.int8)
df['digital'] = df['digital'].astype(np.bool_)
df['category_type'] = df['category_type'].astype(np.int8)
df['category_subtype'] = df['category_subtype'].astype(np.int8)
df['item_cnt_lag_2'] = df['item_cnt_lag_2'].astype(np.int32)
df['item_cnt_lag_3'] = df['item_cnt_lag_3'].astype(np.int32)
df['item_cnt_lag_4'] = df['item_cnt_lag_4'].astype(np.int32)
df['item_cnt_lag_5'] = df['item_cnt_lag_5'].astype(np.int32)
df['item_cnt_lag_6'] = df['item_cnt_lag_6'].astype(np.int32)

for col in df.columns[22:40]:
    df[col] = df[col].astype(np.int32)

df.head()

,shop_id,item_id,month,year,date_block_num,item_cnt,item_price,prev_value,prev_value_year,stock_prev,stock_prev_year,shop_city,shop_category,item_category_id,digital,category_type,category_subtype,item_cnt_lag_2,item_cnt_lag_3,item_cnt_lag_4,item_cnt_lag_5,item_cnt_lag_6,cnt_block_shop_lag_1,cnt_block_shop_lag_2,cnt_block_shop_lag_3,cnt_block_shop_lag_4,cnt_block_shop_lag_5,cnt_block_shop_lag_12,cnt_block_item_lag_1,cnt_block_item_lag_2,cnt_block_item_lag_3,cnt_block_item_lag_4,cnt_block_item_lag_5,cnt_block_item_lag_12,cnt_block_category_lag_1,cnt_block_category_lag_2,cnt_block_category_lag_3,cnt_block_category_lag_4,cnt_block_category_lag_5,cnt_block_category_lag_12,date_item_avg_item_cnt,shop_item_avg_item_cnt,cat_item_avg_item_cnt
0,0,19,1,2013,0,0,0.0,0,0,0.0,0.0,29,4,40,False,7,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.02222,0.0,0.02222
1,0,27,1,2013,0,0,0.0,0,0,0.0,0.0,29,4,19,False,5,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.15550,0.0,0.05682
2,0,28,1,2013,0,0,0.0,0,0,0.0,0.0,29,4,30,False,5,49,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.17770,0.0,0.14120
3,0,29,1,2013,0,0,0.0,0,0,0.0,0.0,29,4,23,False,5,18,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.08887,0.0,0.03738
4,0,32,1,2013,0,6,221.0,0,0,0.0,0.0,29,4,40,False,7,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6.64500,8.0,1.31900


In [ ]:
df = df[df.date_block_num >= 12]
df.reset_index(drop=True, inplace=True)
df.shape

(6425094, 43)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6425094 entries, 0 to 6425093
Data columns (total 43 columns):
 #   Column                     Dtype  
---  ------                     -----  
 0   shop_id                    int8   
 1   item_id                    int16  
 2   month                      int8   
 3   year                       int16  
 4   date_block_num             int8   
 5   item_cnt                   int32  
 6   item_price                 float64
 7   prev_value                 int32  
 8   prev_value_year            int32  
 9   stock_prev                 float64
 10  stock_prev_year            float64
 11  shop_city                  int8   
 12  shop_category              int8   
 13  item_category_id           int8   
 14  digital                    bool   
 15  category_type              int8   
 16  category_subtype           int8   
 17  item_cnt_lag_2             int32  
 18  item_cnt_lag_3             int32  
 19  item_cnt_lag_4             int32  
 20  it

Due to the fact that we predict future, it makes no sense to shuffle the data

In [ ]:
# Indices for RandomizedSearchCV
index = df.index

condition_train = df.date_block_num < 33
train_indices = index[condition_train]

condition_valid = df.date_block_num == 33
valid_indices = index[condition_valid]

train_indices = train_indices.tolist()
valid_indices = valid_indices.tolist()

print(train_indices[-5:])
print(valid_indices[-5:])

[6186917, 6186918, 6186919, 6186920, 6186921]
[6425089, 6425090, 6425091, 6425092, 6425093]


In [ ]:
X = df.drop(['item_cnt'], axis=1)
y = df['item_cnt']

In [ ]:
X_train = df[df.date_block_num < 33].drop(['item_cnt'], axis=1)
y_train = df[df.date_block_num < 33]['item_cnt']
X_valid = df[df.date_block_num == 33].drop(['item_cnt'], axis=1)
y_valid = df[df.date_block_num == 33]['item_cnt']

In [ ]:
from sklearn.model_selection import GridSearchCV

custom_cv = [(train_indices, valid_indices)]
params = {'learning_rate': [0.1, 0.3, 0.5, 1], 
          'subsample': [0.6, 0.7, 0.8, 0.9],
          'max_depth': [8, 9, 10],
          'min_child_weight': [5, 10],
          'colsample_bytree': [0.8, 0.9],
          'n_estimators': [300, 500, 700],
          'gamma': [0.5, 1, 1.5, 2]}

fit_params={"early_stopping_rounds": 30,
            "eval_metric": "rmse", 
            "eval_set": [(X_valid, y_valid)],
            "verbose": 2}

In [ ]:
random_state = 13
verbosity = 2
gp = 'gpu_hist'
obj = 'reg:squarederror'

In [ ]:
xgbr = xgb.XGBRegressor(verbosity=verbosity,
                        objective=obj,
                        tree_method = gp,
                        random_state=random_state)

In [ ]:
r_grid_xgbr = GridSearchCV(xgbr, params,
                           cv=custom_cv,
                           scoring='neg_root_mean_squared_error',
                           verbose=2)

In [ ]:
r_grid_xgbr.fit(X, y, **fit_params)

Выходные данные были обрезаны до нескольких последних строк (5000).
[44]	validation_0-rmse:3.87922
[46]	validation_0-rmse:3.8781
[48]	validation_0-rmse:3.88325
[50]	validation_0-rmse:3.87709
[52]	validation_0-rmse:3.87338
[54]	validation_0-rmse:3.8602
[56]	validation_0-rmse:3.84662
[58]	validation_0-rmse:3.85838
[60]	validation_0-rmse:3.85435
[62]	validation_0-rmse:3.85568
[64]	validation_0-rmse:3.86666
[66]	validation_0-rmse:3.85364
[68]	validation_0-rmse:3.85448
[70]	validation_0-rmse:3.84824
[72]	validation_0-rmse:3.84451
[74]	validation_0-rmse:3.84613
[76]	validation_0-rmse:3.84788
[78]	validation_0-rmse:3.84492
[80]	validation_0-rmse:3.84284
[82]	validation_0-rmse:3.84097
[84]	validation_0-rmse:3.83854
[86]	validation_0-rmse:3.83384
[88]	validation_0-rmse:3.83586
[90]	validation_0-rmse:3.83613
[92]	validation_0-rmse:3.83425
[94]	validation_0-rmse:3.8307
[96]	validation_0-rmse:3.82057
[98]	validation_0-rmse:3.82565
[100]	validation_0-rmse:3.82442
[102]	validation_0-rmse:3.82107
[10

In [ ]:
print("\n The best estimator across ALL searched params:\n", r_grid_xgbr.best_estimator_)
print("\n The best score across ALL searched params:\n", r_grid_xgbr.best_score_)
print("\n The best parameters across ALL searched params:\n", r_grid_xgbr.best_params_)

In [ ]:
xgbr = r_grid_xgbr.best_estimator_

In [ ]:
import matplotlib.pyplot as plt

# retrieve performance metrics
results = xgbr.evals_result()
epochs = len(results['validation_0']['rmse'])
x_axis = range(0, epochs)

# plot rmse
fig, ax = plt.subplots()
ax.plot(x_axis, results['validation_0']['rmse'], label='Train')
# ax.plot(x_axis, results['validation_1']['rmse'], label='Test')
ax.legend()
plt.ylabel('RMSE')
plt.title('XGBoost RMSE')
plt.show()

In [ ]:
from sklearn.metrics import mean_squared_error

X_valid, y_valid = X.iloc[valid_indices], y.iloc[valid_indices]
y_pred = xgbr.predict(X_valid)
y_pred_rounded = np.around(y_pred)
mse = mean_squared_error(y_valid, y_pred)
mse_rd = mean_squared_error(y_valid, y_pred_rounded)

print('MSE: %.2f' % mse)
print('RMSE: %.2f' % (mse ** (0.5)))

print('MSE (rounded): %.2f' % mse_rd)
print('RMSE (rounded): %.2f' % (mse_rd ** (0.5)))